# Automated CV Design: Complete Pipeline Example

This notebook demonstrates a complete workflow for processing pairwise distance data, including:
1. **Data Loading**: Using the refactored `data_access` to load H5/NPY files
2. **Variance Filtering**: Removing low-variance features using knee-point method
3. **Class Assignment**: Assigning 8 classes based on construct × subconstruct combinations
4. **Feature Selection**: Multiple methods (Chi-Squared, Fisher-AMINO, MPSO, BPSO)
5. **Dimensionality Reduction**: Multiple methods (FLDA, GDHLDA, MHLDA, ZHLDA, PCA)

## Data Structure
- **Constructs**: `calmodulin`, `calmodulin-compact` (2 constructs)
- **Subconstructs**: `ca-mg-1-2`, `ca-mg-1-4`, `ca-only`, `mg-only` (4 subconstructs each)
- **Total Classes**: 2 × 4 = 8 classes

## Pipeline Combinations
This framework supports all 20 combinations:
- **Variance**: 1 method
- **Feature Selection**: 4 methods (Chi-Squared, Fisher-AMINO, MPSO, BPSO)
- **Dimensionality Reduction**: 5 methods (FLDA, GDHLDA, MHLDA, ZHLDA, PCA)
- **Total**: 1 × 4 × 5 = 20 pipeline combinations

In [1]:
import sys
import os
import pandas as pd
import numpy as np

# Add the lda directory to Python path
lda_path = os.path.join(os.getcwd(), 'lda')
if lda_path not in sys.path:
    sys.path.append(lda_path)

# Import the interactive pipeline functions
from pipeline_helper import run_interactive_pipeline, create_interactive_pipeline_configs

print("✅ Imports completed successfully!")

✅ Imports completed successfully!


In [2]:
import sys
import os
import pandas as pd
import numpy as np

# Add the lda directory to Python path
lda_path = os.path.join(os.getcwd(), 'lda')
if lda_path not in sys.path:
    sys.path.append(lda_path)

# Import the interactive pipeline functions
from pipeline_helper import run_interactive_pipeline, create_interactive_pipeline_configs

print("✅ Imports completed successfully!")

# =============================================================================
# CELL 2: USE YOUR EXISTING DATA ACCESS
# =============================================================================
# Import your existing data access module
from data_access import create_dataframe_factory, list_available_constructs_subconstructs

# Set the correct data directory for your system
# Update the BASE_DIR to point to your actual data location
import data_access
data_dir = '/work/hdd/bfri/jjeong7/analysis_output/dist_maps'
constructs_dict, subconstructs_dict = list_available_constructs_subconstructs(base_dir=data_dir)

print("📊 Available data:")
print(f"🏗️  Constructs: {list(constructs_dict.keys())}")
print(f"🔧 Subconstructs: {list(subconstructs_dict.keys())}")

# Show details for each construct
for construct, subconstructs in constructs_dict.items():
    print(f"   📋 {construct}: {subconstructs}")

# Create your data factory WITHOUT the class column
# The pipeline runner will inject it later using the callback
data_factory = create_dataframe_factory(base_dir=data_dir)

# Test the data factory
test_data = next(data_factory())
print(f"\n📊 Test data shape: {test_data.shape}")
print(f"🏗️  Constructs: {test_data['construct'].unique()}")
print(f"🔧 Subconstructs: {test_data['subconstruct'].unique()}")
print(f"📋 Feature columns: {len([col for col in test_data.columns if col not in ['construct', 'subconstruct', 'replica', 'frame_number', 'time']])}")

# Show a few columns to understand the data structure
print(f"📋 Sample columns: {test_data.columns.tolist()[:10]}...")


✅ Imports completed successfully!
📊 Available data:
🏗️  Constructs: ['calmodulin', 'calmodulin-zeb-bp2', 'calmodulin-compact']
🔧 Subconstructs: ['ca-mg-1-4', 'mg-only', 'ca-mg-1-2', 'ca-only']
   📋 calmodulin: ['ca-mg-1-2', 'ca-mg-1-4', 'ca-only', 'mg-only']
   📋 calmodulin-zeb-bp2: ['ca-mg-1-2', 'ca-mg-1-4', 'ca-only', 'mg-only']
   📋 calmodulin-compact: ['ca-mg-1-2', 'ca-mg-1-4', 'ca-only', 'mg-only']
Loaded 144 canonical residues from /work/hdd/bfri/jjeong7/analysis_output/dist_maps/calmodulin/ca-mg-1-2/canonical_resids.npy

📊 Test data shape: (3000, 10301)
🏗️  Constructs: ['calmodulin']
🔧 Subconstructs: ['ca-mg-1-2']
📋 Feature columns: 10296
📋 Sample columns: ['RES2_3', 'RES2_4', 'RES2_5', 'RES2_6', 'RES2_7', 'RES2_8', 'RES2_9', 'RES2_10', 'RES2_11', 'RES2_12']...


In [3]:
# Create all possible pipeline combinations
configs = create_interactive_pipeline_configs()

print(f"🔧 Generated {len(configs)} pipeline configurations")
print(f"📊 Feature selection methods: 4 (bpso, mpso, fisher_amino, chi_sq_amino)")
print(f"🎯 Dimensionality reduction methods: 5 (flda, pca, zhlda, mhlda, gdhlda)")
print(f"📈 Total combinations: 4 × 5 = 20")

# Show first few configurations
print("\n📋 First 5 pipeline configurations:")
for i, config in enumerate(configs[:5]):
    print(f"   {i+1}. {config['name']}")

🔧 Generated 20 pipeline configurations
📊 Feature selection methods: 4 (bpso, mpso, fisher_amino, chi_sq_amino)
🎯 Dimensionality reduction methods: 5 (flda, pca, zhlda, mhlda, gdhlda)
📈 Total combinations: 4 × 5 = 20

📋 First 5 pipeline configurations:
   1. bpso_to_flda
   2. bpso_to_pca
   3. bpso_to_zhlda
   4. bpso_to_mhlda
   5. bpso_to_gdhlda


In [ ]:
print("🚀 Starting Interactive Pipeline Runner")
print("💡 Variance runs first, then feature selection, then dimensionality reduction")
print("⚙️  You'll set parameters for each phase")

# Use the pipeline runner
results = run_interactive_pipeline(data_factory, configs)

🚀 Starting Interactive Pipeline Runner
💡 Variance runs first, then feature selection, then dimensionality reduction
⚙️  You'll set parameters for each phase


Found cached result for VARIANCE (pipeline_cache/variance.pkl). Load? (Y/n):  y


Loaded cached VARIANCE data.
Shape: (119991, 274)

[FEATURE_SELECTION : CHI_SQ_AMINO]
  max_amino: 10
  q_bins: 5
  knee_S: 5.0
  sample_rows: 20000



Modify? (y/N):  n


Running CHI_SQ_AMINO...
Successfully imported AMINO from: /u/jjeong7/ondemand/lda/feature_selection/amino.py
Sampling to estimate Chi-Squared bin edges and classes...
Building Chi-Squared scores sequentially...
Knee Point: 16856.4120 | Candidates: 29
Loading 29 candidate features into memory for AMINO...
Running AMINO: Asking for 10 features from 29 candidates...
Selected bandwidth: 0.3132549209074307

Checking 10 order parameters...
Error executing CHI_SQ_AMINO: 'NoneType' object cannot be interpreted as an integer


In [ ]:
print(f"📊 Successful pipelines: {len(results)}")

if results:
    print("\n📈 Results Summary:")
    for pipeline_name, result in results.items():
        final_df = result['final_result']
        feature_cols = [col for col in final_df.columns if col != 'class']
        
        print(f"   ✅ {pipeline_name}:")
        print(f"      📏 Shape: {final_df.shape}")
        print(f"      🔧 Features: {len(feature_cols)}")
        print(f"      🏷️  Classes: {final_df['class'].nunique()}")
        
        # Show feature names if not too many
        if len(feature_cols) <= 5:
            print(f"      📋 Features: {feature_cols}")
        else:
            print(f"      📋 Features: {feature_cols[:3]}...{feature_cols[-2:]}")
        print()
else:
    print("⚠️  No pipelines completed successfully")

In [ ]:
if results:
    import pickle
    import datetime
    
    # Create timestamp for filename
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_file = f"pipeline_results_{timestamp}.pkl"
    
    # Save results to file
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"💾 Results saved to: {results_file}")
    
    # Also save summary to CSV
    summary_data = []
    for pipeline_name, result in results.items():
        final_df = result['final_result']
        summary_data.append({
            'pipeline': pipeline_name,
            'samples': final_df.shape[0],
            'features': len([col for col in final_df.columns if col != 'class']),
            'classes': final_df['class'].nunique(),
            'feature_selection': result['config']['feature_selection'],
            'dimensionality_reduction': result['config']['dimensionality_reduction']
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_file = f"pipeline_summary_{timestamp}.csv"
    summary_df.to_csv(summary_file, index=False)
    print(f"📊 Summary saved to: {summary_file}")
else:
    print("⚠️  No results to save")